# Advanced Decorators in Python

---

## Table of Contents
1. Decorator Review
2. Decorators with Arguments
3. Class-Based Decorators
4. Decorating Classes
5. Stacking Decorators
6. Preserving Metadata with functools.wraps
7. Decorator Factories
8. Stateful Decorators
9. Optional Arguments Pattern
10. Built-in Decorators Deep Dive
11. Practical Decorator Patterns
12. Key Points
13. Practice Exercises

---

## 1. Decorator Review

In [2]:
# Basic decorator pattern
def simple_decorator(func):
    def wrapper(*args, **kwargs):
        print("Before function call")
        result = func(*args, **kwargs)
        print("After function call")
        return result
    return wrapper

@simple_decorator
def greet(name):
    """Greet someone by name."""
    print(f"Hello, {name}!")

greet("Alice")

Before function call
Hello, Alice!
After function call


In [3]:
# Equivalent to:
def greet_without_syntax(name):
    print(f"Hello, {name}!")

greet_manual = simple_decorator(greet_without_syntax)
greet_manual("Bob")

Before function call
Hello, Bob!
After function call


---

## 2. Decorators with Arguments

When a decorator needs arguments, we need an extra layer of nesting.

In [4]:
# Decorator with arguments - triple nested functions
def repeat(times):
    """Decorator that repeats function execution."""
    def decorator(func):
        def wrapper(*args, **kwargs):
            for _ in range(times):
                result = func(*args, **kwargs)
            return result
        return wrapper
    return decorator

@repeat(times=3)
def say_hello():
    print("Hello!")

say_hello()

Hello!
Hello!
Hello!


In [5]:
# Understanding the call order:
# @repeat(times=3) is equivalent to:
# 1. decorator = repeat(times=3)  # Returns decorator function
# 2. say_hello = decorator(say_hello)  # Returns wrapper function

# More practical example: retry decorator
import time

def retry(max_attempts=3, delay=1):
    """Retry function on exception."""
    def decorator(func):
        def wrapper(*args, **kwargs):
            last_exception = None
            for attempt in range(1, max_attempts + 1):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    last_exception = e
                    print(f"Attempt {attempt} failed: {e}")
                    if attempt < max_attempts:
                        time.sleep(delay)
            raise last_exception
        return wrapper
    return decorator

@retry(max_attempts=3, delay=0.1)
def flaky_function():
    import random
    if random.random() < 0.7:
        raise ValueError("Random failure!")
    return "Success!"

try:
    result = flaky_function()
    print(f"Result: {result}")
except ValueError as e:
    print(f"All attempts failed: {e}")

Attempt 1 failed: Random failure!
Attempt 2 failed: Random failure!
Attempt 3 failed: Random failure!
All attempts failed: Random failure!


In [6]:
# Decorator with keyword-only arguments
def validate_types(**type_hints):
    """Validate argument types at runtime."""
    def decorator(func):
        def wrapper(*args, **kwargs):
            # Get function parameter names
            import inspect
            sig = inspect.signature(func)
            params = list(sig.parameters.keys())
            
            # Map args to parameter names
            all_args = dict(zip(params, args))
            all_args.update(kwargs)
            
            # Validate types
            for param, expected_type in type_hints.items():
                if param in all_args:
                    value = all_args[param]
                    if not isinstance(value, expected_type):
                        raise TypeError(
                            f"{param} must be {expected_type.__name__}, "
                            f"got {type(value).__name__}"
                        )
            
            return func(*args, **kwargs)
        return wrapper
    return decorator

@validate_types(name=str, age=int)
def create_user(name, age):
    return {"name": name, "age": age}

print(create_user("Alice", 30))

try:
    create_user("Bob", "thirty")  # Wrong type
except TypeError as e:
    print(f"Error: {e}")

{'name': 'Alice', 'age': 30}
Error: age must be int, got str


---

## 3. Class-Based Decorators

Classes with `__call__` can be used as decorators.

In [7]:
# Basic class-based decorator
class CountCalls:
    """Count how many times a function is called."""
    
    def __init__(self, func):
        self.func = func
        self.count = 0
    
    def __call__(self, *args, **kwargs):
        self.count += 1
        print(f"Call #{self.count} to {self.func.__name__}")
        return self.func(*args, **kwargs)

@CountCalls
def say_hi():
    print("Hi!")

say_hi()
say_hi()
say_hi()
print(f"Total calls: {say_hi.count}")

Call #1 to say_hi
Hi!
Call #2 to say_hi
Hi!
Call #3 to say_hi
Hi!
Total calls: 3


In [8]:
# Class decorator with arguments
class Timer:
    """Time function execution with optional label."""
    
    def __init__(self, label=None):
        self.label = label
        self.func = None
    
    def __call__(self, *args, **kwargs):
        # If first arg is callable, we're being used without parens
        if self.func is None and len(args) == 1 and callable(args[0]):
            self.func = args[0]
            return self
        
        import time
        start = time.time()
        result = self.func(*args, **kwargs)
        elapsed = time.time() - start
        
        label = self.label or self.func.__name__
        print(f"[{label}] Took {elapsed:.4f}s")
        return result

@Timer(label="slow_operation")
def slow_func():
    import time
    time.sleep(0.1)
    return "Done"

slow_func()

[slow_operation] Took 0.1005s


'Done'

In [9]:
# Class decorator that maintains state
class Memoize:
    """Cache function results."""
    
    def __init__(self, func):
        self.func = func
        self.cache = {}
    
    def __call__(self, *args):
        if args not in self.cache:
            self.cache[args] = self.func(*args)
        return self.cache[args]
    
    def clear_cache(self):
        self.cache.clear()
    
    def cache_info(self):
        return f"Cache size: {len(self.cache)}"

@Memoize
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 1) + fibonacci(n - 2)

print(f"fib(30) = {fibonacci(30)}")
print(fibonacci.cache_info())

fib(30) = 832040
Cache size: 31


---

## 4. Decorating Classes

In [10]:
# Decorator that adds methods to a class
def add_repr(cls):
    """Add a __repr__ method based on __init__ parameters."""
    import inspect
    
    sig = inspect.signature(cls.__init__)
    params = [p for p in sig.parameters.keys() if p != 'self']
    
    def __repr__(self):
        values = [f"{p}={getattr(self, p)!r}" for p in params]
        return f"{cls.__name__}({', '.join(values)})"
    
    cls.__repr__ = __repr__
    return cls

@add_repr
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

p = Point(3, 4)
print(p)  # Point(x=3, y=4)

Point(x=3, y=4)


In [11]:
# Singleton decorator
def singleton(cls):
    """Ensure only one instance of class exists."""
    instances = {}
    
    def get_instance(*args, **kwargs):
        if cls not in instances:
            instances[cls] = cls(*args, **kwargs)
        return instances[cls]
    
    return get_instance

@singleton
class Database:
    def __init__(self, connection_string):
        print(f"Connecting to {connection_string}...")
        self.connection_string = connection_string

db1 = Database("postgres://localhost")
db2 = Database("mysql://localhost")  # Ignored, returns same instance

print(f"db1 is db2: {db1 is db2}")
print(f"Connection: {db2.connection_string}")

Connecting to postgres://localhost...
db1 is db2: True
Connection: postgres://localhost


In [12]:
# Decorator that adds validation
def validate_attributes(**validators):
    """Add attribute validation to class."""
    def decorator(cls):
        original_init = cls.__init__
        
        def new_init(self, *args, **kwargs):
            original_init(self, *args, **kwargs)
            
            for attr, validator in validators.items():
                if hasattr(self, attr):
                    value = getattr(self, attr)
                    if not validator(value):
                        raise ValueError(f"Invalid value for {attr}: {value}")
        
        cls.__init__ = new_init
        return cls
    return decorator

@validate_attributes(
    age=lambda x: x >= 0,
    email=lambda x: '@' in x
)
class User:
    def __init__(self, name, age, email):
        self.name = name
        self.age = age
        self.email = email

user = User("Alice", 30, "alice@example.com")
print(f"Created user: {user.name}")

try:
    invalid_user = User("Bob", -5, "bob@example.com")
except ValueError as e:
    print(f"Validation error: {e}")

Created user: Alice
Validation error: Invalid value for age: -5


In [13]:
# Register classes with a decorator
class PluginRegistry:
    plugins = {}
    
    @classmethod
    def register(cls, name):
        def decorator(plugin_cls):
            cls.plugins[name] = plugin_cls
            return plugin_cls
        return decorator
    
    @classmethod
    def get(cls, name):
        return cls.plugins.get(name)

@PluginRegistry.register("json")
class JSONParser:
    def parse(self, data):
        import json
        return json.loads(data)

@PluginRegistry.register("csv")
class CSVParser:
    def parse(self, data):
        return [line.split(',') for line in data.split('\n')]

print(f"Registered plugins: {list(PluginRegistry.plugins.keys())}")

parser = PluginRegistry.get("json")()
print(parser.parse('{"key": "value"}'))

Registered plugins: ['json', 'csv']
{'key': 'value'}


---

## 5. Stacking Decorators

In [14]:
# Order matters! Decorators apply bottom-up
def decorator_a(func):
    def wrapper(*args, **kwargs):
        print("A: before")
        result = func(*args, **kwargs)
        print("A: after")
        return result
    return wrapper

def decorator_b(func):
    def wrapper(*args, **kwargs):
        print("B: before")
        result = func(*args, **kwargs)
        print("B: after")
        return result
    return wrapper

@decorator_a
@decorator_b
def greet():
    print("Hello!")

# Same as: greet = decorator_a(decorator_b(greet))
print("Calling greet():")
greet()

Calling greet():
A: before
B: before
Hello!
B: after
A: after


In [15]:
# Practical stacking: logging + timing + authentication
from functools import wraps
import time

def log_call(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(f"LOG: Calling {func.__name__}")
        return func(*args, **kwargs)
    return wrapper

def time_it(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        print(f"TIME: {func.__name__} took {time.time() - start:.4f}s")
        return result
    return wrapper

def require_auth(func):
    @wraps(func)
    def wrapper(user, *args, **kwargs):
        if not user.get('authenticated', False):
            raise PermissionError("User not authenticated")
        return func(user, *args, **kwargs)
    return wrapper

@log_call
@time_it
@require_auth
def get_sensitive_data(user):
    time.sleep(0.05)
    return f"Secret data for {user['name']}"

authenticated_user = {'name': 'Alice', 'authenticated': True}
result = get_sensitive_data(authenticated_user)
print(f"Result: {result}")

LOG: Calling get_sensitive_data
TIME: get_sensitive_data took 0.0502s
Result: Secret data for Alice


---

## 6. Preserving Metadata with functools.wraps

In [16]:
# Problem: decorators lose function metadata
def bad_decorator(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

@bad_decorator
def documented_function():
    """This is the docstring."""
    pass

print(f"Name: {documented_function.__name__}")  # 'wrapper' - wrong!
print(f"Doc: {documented_function.__doc__}")    # None - wrong!

Name: wrapper
Doc: None


In [17]:
# Solution: use functools.wraps
from functools import wraps

def good_decorator(func):
    @wraps(func)  # Copies metadata from func to wrapper
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

@good_decorator
def documented_function():
    """This is the docstring."""
    pass

print(f"Name: {documented_function.__name__}")  # 'documented_function'
print(f"Doc: {documented_function.__doc__}")    # 'This is the docstring.'

Name: documented_function
Doc: This is the docstring.


In [18]:
# What wraps preserves
from functools import WRAPPER_ASSIGNMENTS, WRAPPER_UPDATES

print(f"Assignments copied: {WRAPPER_ASSIGNMENTS}")
print(f"Updates merged: {WRAPPER_UPDATES}")

Assignments copied: ('__module__', '__name__', '__qualname__', '__doc__', '__annotations__', '__type_params__')
Updates merged: ('__dict__',)


In [19]:
# Accessing the original function
def my_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

@my_decorator
def original():
    """Original function."""
    return "original"

# __wrapped__ attribute gives access to original
print(f"Wrapped function: {original}")
print(f"Original function: {original.__wrapped__}")
print(f"Same function? {original.__wrapped__ is not original}")

Wrapped function: <function original at 0x00000178AACA9C60>
Original function: <function original at 0x00000178AACA98A0>
Same function? True


---

## 7. Decorator Factories

In [20]:
# Factory that creates specialized decorators
def make_limiter(limit_type):
    """Create rate limiters of different types."""
    
    if limit_type == "calls":
        def decorator(max_calls):
            def inner_decorator(func):
                calls = [0]
                @wraps(func)
                def wrapper(*args, **kwargs):
                    if calls[0] >= max_calls:
                        raise RuntimeError(f"Max {max_calls} calls exceeded")
                    calls[0] += 1
                    return func(*args, **kwargs)
                return wrapper
            return inner_decorator
        return decorator
    
    elif limit_type == "time":
        def decorator(max_seconds):
            def inner_decorator(func):
                @wraps(func)
                def wrapper(*args, **kwargs):
                    import signal
                    # Simplified - just warn if takes too long
                    start = time.time()
                    result = func(*args, **kwargs)
                    if time.time() - start > max_seconds:
                        print(f"Warning: {func.__name__} exceeded {max_seconds}s")
                    return result
                return wrapper
            return inner_decorator
        return decorator

# Create specialized decorators
limit_calls = make_limiter("calls")
limit_time = make_limiter("time")

@limit_calls(3)
def limited_function():
    print("Called!")

for i in range(4):
    try:
        limited_function()
    except RuntimeError as e:
        print(f"Error: {e}")

Called!
Called!
Called!
Error: Max 3 calls exceeded


In [21]:
# Decorator factory with configuration
class LoggerFactory:
    """Factory for creating logging decorators."""
    
    def __init__(self, default_level="INFO"):
        self.default_level = default_level
    
    def log(self, level=None, include_args=False):
        level = level or self.default_level
        
        def decorator(func):
            @wraps(func)
            def wrapper(*args, **kwargs):
                msg = f"[{level}] Calling {func.__name__}"
                if include_args:
                    msg += f" with args={args}, kwargs={kwargs}"
                print(msg)
                return func(*args, **kwargs)
            return wrapper
        return decorator

# Create a logger factory
logger = LoggerFactory(default_level="DEBUG")

@logger.log()
def func1():
    pass

@logger.log(level="ERROR", include_args=True)
def func2(x, y):
    pass

func1()
func2(1, 2)

[DEBUG] Calling func1
[ERROR] Calling func2 with args=(1, 2), kwargs={}


---

## 8. Stateful Decorators

In [22]:
# Decorator that tracks statistics
from functools import wraps
import time

def track_stats(func):
    """Track call statistics for a function."""
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        try:
            result = func(*args, **kwargs)
            wrapper.success_count += 1
            return result
        except Exception as e:
            wrapper.error_count += 1
            raise
        finally:
            wrapper.total_time += time.time() - start
            wrapper.call_count += 1
    
    # Initialize state
    wrapper.call_count = 0
    wrapper.success_count = 0
    wrapper.error_count = 0
    wrapper.total_time = 0
    
    def get_stats():
        return {
            'calls': wrapper.call_count,
            'successes': wrapper.success_count,
            'errors': wrapper.error_count,
            'avg_time': wrapper.total_time / max(wrapper.call_count, 1)
        }
    
    wrapper.get_stats = get_stats
    return wrapper

@track_stats
def process_data(data):
    time.sleep(0.01)
    if data < 0:
        raise ValueError("Negative data")
    return data * 2

# Run some calls
for i in range(-2, 5):
    try:
        process_data(i)
    except ValueError:
        pass

print(f"Stats: {process_data.get_stats()}")

Stats: {'calls': 7, 'successes': 5, 'errors': 2, 'avg_time': 0.010310377393450056}


In [23]:
# Decorator with persistent state across decorated functions
class GlobalTracker:
    """Track all decorated function calls globally."""
    
    _all_calls = []
    
    def __init__(self, func):
        self.func = func
        wraps(func)(self)
    
    def __call__(self, *args, **kwargs):
        GlobalTracker._all_calls.append({
            'function': self.func.__name__,
            'args': args,
            'time': time.time()
        })
        return self.func(*args, **kwargs)
    
    @classmethod
    def get_all_calls(cls):
        return cls._all_calls.copy()
    
    @classmethod
    def clear_history(cls):
        cls._all_calls.clear()

@GlobalTracker
def func_a(x):
    return x + 1

@GlobalTracker
def func_b(x):
    return x * 2

func_a(1)
func_b(2)
func_a(3)

print("All tracked calls:")
for call in GlobalTracker.get_all_calls():
    print(f"  {call['function']}({call['args']})")

All tracked calls:
  func_a((1,))
  func_b((2,))
  func_a((3,))


---

## 9. Optional Arguments Pattern

Create decorators that work with or without parentheses.

In [24]:
# Pattern for optional decorator arguments
def debug(func=None, *, prefix='DEBUG'):
    """Debug decorator that works with or without arguments.
    
    Usage:
        @debug           # Without arguments
        @debug()         # With empty parentheses
        @debug(prefix='INFO')  # With arguments
    """
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print(f"[{prefix}] Calling {func.__name__}")
            result = func(*args, **kwargs)
            print(f"[{prefix}] {func.__name__} returned {result}")
            return result
        return wrapper
    
    if func is not None:
        # Called without arguments: @debug
        return decorator(func)
    else:
        # Called with arguments: @debug() or @debug(prefix='INFO')
        return decorator

@debug
def add(a, b):
    return a + b

@debug(prefix='TRACE')
def multiply(a, b):
    return a * b

print("\nCalling add:")
add(2, 3)

print("\nCalling multiply:")
multiply(4, 5)


Calling add:
[DEBUG] Calling add
[DEBUG] add returned 5

Calling multiply:
[TRACE] Calling multiply
[TRACE] multiply returned 20


20

In [25]:
# Alternative pattern using a class
class flexible_decorator:
    """Decorator that works with or without arguments."""
    
    def __init__(self, func=None, *, message="Called"):
        self.message = message
        self.func = func
        if func is not None:
            wraps(func)(self)
    
    def __call__(self, *args, **kwargs):
        if self.func is None:
            # Used as @decorator() with arguments
            func = args[0]
            self.func = func
            wraps(func)(self)
            return self
        
        print(f"{self.message}: {self.func.__name__}")
        return self.func(*args, **kwargs)

@flexible_decorator
def foo():
    return "foo"

@flexible_decorator(message="Executing")
def bar():
    return "bar"

print(foo())
print(bar())

Called: foo
foo
Executing: bar
bar


---

## 10. Built-in Decorators Deep Dive

In [26]:
# functools.lru_cache - memoization with LRU eviction
from functools import lru_cache

@lru_cache(maxsize=128)
def expensive_computation(n):
    """Simulate expensive computation."""
    print(f"Computing for {n}...")
    return n ** 2

print(expensive_computation(4))  # Computes
print(expensive_computation(4))  # Cached
print(expensive_computation(5))  # Computes
print(expensive_computation(4))  # Cached

print(f"\nCache info: {expensive_computation.cache_info()}")
expensive_computation.cache_clear()
print(f"After clear: {expensive_computation.cache_info()}")

Computing for 4...
16
16
Computing for 5...
25
16

Cache info: CacheInfo(hits=2, misses=2, maxsize=128, currsize=2)
After clear: CacheInfo(hits=0, misses=0, maxsize=128, currsize=0)


In [27]:
# functools.singledispatch - generic functions
from functools import singledispatch

@singledispatch
def process(data):
    """Default implementation."""
    return f"Unknown type: {type(data).__name__}"

@process.register(int)
def _(data):
    return f"Integer: {data * 2}"

@process.register(str)
def _(data):
    return f"String: {data.upper()}"

@process.register(list)
def _(data):
    return f"List with {len(data)} items"

print(process(42))
print(process("hello"))
print(process([1, 2, 3]))
print(process({"key": "value"}))

Integer: 84
String: HELLO
List with 3 items
Unknown type: dict


In [28]:
# functools.total_ordering - fill in comparison methods
from functools import total_ordering

@total_ordering
class Version:
    """Version class with complete ordering from just __eq__ and __lt__."""
    
    def __init__(self, major, minor, patch):
        self.major = major
        self.minor = minor
        self.patch = patch
    
    def __eq__(self, other):
        return (self.major, self.minor, self.patch) == \
               (other.major, other.minor, other.patch)
    
    def __lt__(self, other):
        return (self.major, self.minor, self.patch) < \
               (other.major, other.minor, other.patch)
    
    def __repr__(self):
        return f"Version({self.major}.{self.minor}.{self.patch})"

v1 = Version(1, 0, 0)
v2 = Version(1, 2, 3)
v3 = Version(2, 0, 0)

print(f"v1 < v2: {v1 < v2}")
print(f"v2 <= v3: {v2 <= v3}")
print(f"v3 > v1: {v3 > v1}")
print(f"v1 >= v1: {v1 >= v1}")

v1 < v2: True
v2 <= v3: True
v3 > v1: True
v1 >= v1: True


In [29]:
# dataclasses.dataclass - auto-generate special methods
from dataclasses import dataclass, field
from typing import List

@dataclass(order=True)
class Product:
    """Product with auto-generated methods."""
    name: str
    price: float
    quantity: int = 0
    tags: List[str] = field(default_factory=list)
    
    # Custom method
    def total_value(self):
        return self.price * self.quantity

p1 = Product("Apple", 1.50, 10)
p2 = Product("Apple", 1.50, 10)
p3 = Product("Banana", 0.75, 20)

print(f"p1: {p1}")
print(f"p1 == p2: {p1 == p2}")
print(f"p1 < p3: {p1 < p3}")
print(f"p1.total_value(): {p1.total_value()}")

p1: Product(name='Apple', price=1.5, quantity=10, tags=[])
p1 == p2: True
p1 < p3: True
p1.total_value(): 15.0


---

## 11. Practical Decorator Patterns

In [30]:
# Pattern 1: Caching with expiration
import time
from functools import wraps

def cache_with_ttl(ttl_seconds=60):
    """Cache results with time-to-live expiration."""
    def decorator(func):
        cache = {}
        
        @wraps(func)
        def wrapper(*args):
            now = time.time()
            
            # Check if cached and not expired
            if args in cache:
                result, timestamp = cache[args]
                if now - timestamp < ttl_seconds:
                    print(f"  [Cache hit for {args}]")
                    return result
            
            # Compute and cache
            print(f"  [Computing for {args}]")
            result = func(*args)
            cache[args] = (result, now)
            return result
        
        wrapper.clear_cache = lambda: cache.clear()
        return wrapper
    return decorator

@cache_with_ttl(ttl_seconds=1)
def get_data(key):
    return f"Data for {key}"

print(get_data("a"))  # Computes
print(get_data("a"))  # Cached
time.sleep(1.1)
print(get_data("a"))  # Expired, recomputes

  [Computing for ('a',)]
Data for a
  [Cache hit for ('a',)]
Data for a
  [Computing for ('a',)]
Data for a


In [31]:
# Pattern 2: Pre/Post condition checking
def contract(pre=None, post=None):
    """Design by contract decorator."""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Check precondition
            if pre is not None:
                assert pre(*args, **kwargs), \
                    f"Precondition failed for {func.__name__}"
            
            result = func(*args, **kwargs)
            
            # Check postcondition
            if post is not None:
                assert post(result), \
                    f"Postcondition failed for {func.__name__}"
            
            return result
        return wrapper
    return decorator

@contract(
    pre=lambda x: x >= 0,        # x must be non-negative
    post=lambda r: r >= 0        # result must be non-negative
)
def square_root(x):
    return x ** 0.5

print(f"sqrt(16) = {square_root(16)}")

try:
    square_root(-4)
except AssertionError as e:
    print(f"Contract violation: {e}")

sqrt(16) = 4.0
Contract violation: Precondition failed for square_root


In [32]:
# Pattern 3: Dependency injection
def inject(**dependencies):
    """Inject dependencies into function."""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Merge injected dependencies with kwargs
            merged_kwargs = {**dependencies, **kwargs}
            return func(*args, **merged_kwargs)
        return wrapper
    return decorator

# Mock services
class Logger:
    def log(self, msg): print(f"LOG: {msg}")

class Database:
    def query(self, sql): return f"Results for: {sql}"

# Inject dependencies
@inject(logger=Logger(), db=Database())
def process_request(request_id, logger, db):
    logger.log(f"Processing {request_id}")
    return db.query(f"SELECT * FROM requests WHERE id={request_id}")

result = process_request("REQ-001")  # No need to pass logger and db
print(result)

LOG: Processing REQ-001
Results for: SELECT * FROM requests WHERE id=REQ-001


In [33]:
# Pattern 4: Circuit breaker
class CircuitBreaker:
    """Prevent cascading failures with circuit breaker pattern."""
    
    def __init__(self, failure_threshold=5, reset_timeout=30):
        self.failure_threshold = failure_threshold
        self.reset_timeout = reset_timeout
        self.failures = 0
        self.last_failure_time = 0
        self.state = "closed"  # closed, open, half-open
    
    def __call__(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Check if circuit is open
            if self.state == "open":
                if time.time() - self.last_failure_time > self.reset_timeout:
                    self.state = "half-open"
                else:
                    raise RuntimeError("Circuit breaker is OPEN")
            
            try:
                result = func(*args, **kwargs)
                # Success - reset if half-open
                if self.state == "half-open":
                    self.state = "closed"
                    self.failures = 0
                return result
            except Exception as e:
                self.failures += 1
                self.last_failure_time = time.time()
                
                if self.failures >= self.failure_threshold:
                    self.state = "open"
                    print(f"Circuit breaker OPENED after {self.failures} failures")
                
                raise
        
        wrapper.get_state = lambda: self.state
        return wrapper

@CircuitBreaker(failure_threshold=3, reset_timeout=1)
def unreliable_service():
    import random
    if random.random() < 0.8:
        raise ConnectionError("Service unavailable")
    return "Success!"

for i in range(6):
    try:
        print(f"Attempt {i+1}: {unreliable_service()}")
    except Exception as e:
        print(f"Attempt {i+1}: {type(e).__name__}: {e}")

print(f"\nCircuit state: {unreliable_service.get_state()}")

Attempt 1: ConnectionError: Service unavailable
Attempt 2: ConnectionError: Service unavailable
Circuit breaker OPENED after 3 failures
Attempt 3: ConnectionError: Service unavailable
Attempt 4: RuntimeError: Circuit breaker is OPEN
Attempt 5: RuntimeError: Circuit breaker is OPEN
Attempt 6: RuntimeError: Circuit breaker is OPEN

Circuit state: open


In [34]:
# Pattern 5: Method chaining helper
def chainable(func):
    """Make method return self for chaining."""
    @wraps(func)
    def wrapper(self, *args, **kwargs):
        func(self, *args, **kwargs)
        return self
    return wrapper

class QueryBuilder:
    def __init__(self):
        self._select = []
        self._from = None
        self._where = []
    
    @chainable
    def select(self, *columns):
        self._select.extend(columns)
    
    @chainable
    def from_table(self, table):
        self._from = table
    
    @chainable
    def where(self, condition):
        self._where.append(condition)
    
    def build(self):
        query = f"SELECT {', '.join(self._select)} FROM {self._from}"
        if self._where:
            query += f" WHERE {' AND '.join(self._where)}"
        return query

query = (QueryBuilder()
         .select("id", "name", "email")
         .from_table("users")
         .where("active = true")
         .where("age > 18")
         .build())

print(query)

SELECT id, name, email FROM users WHERE active = true AND age > 18


---

## 12. Key Points

1. **Basic Pattern**: Decorator wraps function, returns new callable
2. **With Arguments**: Need extra nesting (3 levels)
3. **Class Decorators**: Use `__call__` for callable instances
4. **Class Decoration**: Modify or wrap entire classes
5. **Stacking**: Applied bottom-up, executed top-down
6. **functools.wraps**: Always use to preserve metadata
7. **Factories**: Create specialized decorators dynamically
8. **Stateful**: Use function attributes or class instance variables
9. **Optional Args**: Pattern to support @dec and @dec()
10. **Built-ins**: lru_cache, singledispatch, total_ordering, dataclass

---

## 13. Practice Exercises

In [35]:
# Exercise 1: Create a @throttle decorator
# - Limits how often a function can be called
# - If called too soon, return last result
# - throttle(seconds=5) means min 5 seconds between calls

def throttle(seconds):
    pass

# Test:
# @throttle(seconds=1)
# def get_price():
#     return random.uniform(100, 200)
# 
# print(get_price())  # Actually calls
# print(get_price())  # Returns cached (called too soon)
# time.sleep(1.1)
# print(get_price())  # Actually calls again

In [36]:
# Exercise 2: Create a @deprecated decorator
# - Warns when decorated function is used
# - Optional: specify replacement function name
# - Optional: specify version when it will be removed

def deprecated(reason=None, replacement=None, remove_in=None):
    pass

# Test:
# @deprecated(replacement="new_function", remove_in="2.0")
# def old_function():
#     return "old result"
# 
# old_function()  # Should warn about deprecation

In [37]:
# Exercise 3: Create a @once_per decorator
# - Ensures function runs at most once per unique argument
# - Subsequent calls with same args are silently ignored
# - Different from caching - doesn't return cached value

def once_per(*key_args):
    pass

# Test:
# @once_per('user_id')
# def send_welcome_email(user_id, email):
#     print(f"Sending welcome email to {email}")
# 
# send_welcome_email(1, "a@test.com")  # Sends
# send_welcome_email(1, "a@test.com")  # Ignored (same user_id)
# send_welcome_email(2, "b@test.com")  # Sends (different user_id)

In [38]:
# Exercise 4: Create a @fallback decorator
# - If decorated function raises exception, call fallback function
# - Fallback receives same arguments

def fallback(fallback_func):
    pass

# Test:
# def default_data(key):
#     return f"Default for {key}"
# 
# @fallback(default_data)
# def fetch_data(key):
#     if key == "missing":
#         raise KeyError("Not found")
#     return f"Data for {key}"
# 
# print(fetch_data("existing"))  # "Data for existing"
# print(fetch_data("missing"))   # "Default for missing"

In [39]:
# Exercise 5: Create a @trace decorator
# - Logs function entry with args
# - Logs function exit with result or exception
# - Tracks call depth for nested calls
# - Indents output based on depth

def trace(func):
    pass

# Test:
# @trace
# def factorial(n):
#     if n <= 1:
#         return 1
#     return n * factorial(n - 1)
# 
# factorial(4)
# Output:
# --> factorial(4)
#     --> factorial(3)
#         --> factorial(2)
#             --> factorial(1)
#             <-- factorial = 1
#         <-- factorial = 2
#     <-- factorial = 6
# <-- factorial = 24

---

## Solutions

In [40]:
# Solution 1:
import time
from functools import wraps

def throttle(seconds):
    def decorator(func):
        last_call = [0]
        last_result = [None]
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            now = time.time()
            if now - last_call[0] >= seconds:
                last_call[0] = now
                last_result[0] = func(*args, **kwargs)
            return last_result[0]
        
        return wrapper
    return decorator

import random

@throttle(seconds=1)
def get_price():
    return random.uniform(100, 200)

print(f"Call 1: {get_price():.2f}")
print(f"Call 2: {get_price():.2f}")  # Same as call 1
time.sleep(1.1)
print(f"Call 3: {get_price():.2f}")  # New value

Call 1: 177.25
Call 2: 177.25
Call 3: 153.29


In [41]:
# Solution 2:
import warnings
from functools import wraps

def deprecated(reason=None, replacement=None, remove_in=None):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            msg = f"{func.__name__} is deprecated."
            if reason:
                msg += f" Reason: {reason}"
            if replacement:
                msg += f" Use {replacement} instead."
            if remove_in:
                msg += f" Will be removed in version {remove_in}."
            
            warnings.warn(msg, DeprecationWarning, stacklevel=2)
            return func(*args, **kwargs)
        return wrapper
    return decorator

@deprecated(replacement="new_function", remove_in="2.0")
def old_function():
    return "old result"

# Enable deprecation warnings for demo
warnings.simplefilter('always', DeprecationWarning)
result = old_function()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_32920\1581802725.py:28: DeprecationWarning: old_function is deprecated. Use new_function instead. Will be removed in version 2.0.
  result = old_function()


In [42]:
# Solution 3:
from functools import wraps
import inspect

def once_per(*key_args):
    def decorator(func):
        seen = set()
        sig = inspect.signature(func)
        
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Bind arguments to get parameter names
            bound = sig.bind(*args, **kwargs)
            bound.apply_defaults()
            
            # Create key from specified arguments
            key = tuple(bound.arguments.get(k) for k in key_args)
            
            if key in seen:
                return None
            
            seen.add(key)
            return func(*args, **kwargs)
        
        wrapper.reset = lambda: seen.clear()
        return wrapper
    return decorator

@once_per('user_id')
def send_welcome_email(user_id, email):
    print(f"Sending welcome email to {email}")

send_welcome_email(1, "a@test.com")  # Sends
send_welcome_email(1, "a@test.com")  # Ignored
send_welcome_email(2, "b@test.com")  # Sends

Sending welcome email to a@test.com
Sending welcome email to b@test.com


In [43]:
# Solution 4:
from functools import wraps

def fallback(fallback_func):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            try:
                return func(*args, **kwargs)
            except Exception:
                return fallback_func(*args, **kwargs)
        return wrapper
    return decorator

def default_data(key):
    return f"Default for {key}"

@fallback(default_data)
def fetch_data(key):
    if key == "missing":
        raise KeyError("Not found")
    return f"Data for {key}"

print(fetch_data("existing"))
print(fetch_data("missing"))

Data for existing
Default for missing


In [44]:
# Solution 5:
from functools import wraps

def trace(func):
    depth = [0]  # Track nesting depth
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        indent = "    " * depth[0]
        args_str = ", ".join(repr(a) for a in args)
        print(f"{indent}--> {func.__name__}({args_str})")
        
        depth[0] += 1
        try:
            result = func(*args, **kwargs)
            print(f"{indent}<-- {func.__name__} = {result}")
            return result
        except Exception as e:
            print(f"{indent}<-- {func.__name__} raised {type(e).__name__}: {e}")
            raise
        finally:
            depth[0] -= 1
    
    return wrapper

@trace
def factorial(n):
    if n <= 1:
        return 1
    return n * factorial(n - 1)

factorial(4)

--> factorial(4)
    --> factorial(3)
        --> factorial(2)
            --> factorial(1)
            <-- factorial = 1
        <-- factorial = 2
    <-- factorial = 6
<-- factorial = 24


24